In [ ]:
# uv venv dlt-hw --python 3.10

# python -m ensurepip --default-pip
import requests
import dlt
from dlt.destinations import qdrant

In [ ]:
dlt.__version__ # '1.14.1'

In [ ]:
# dlt rosource 
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [ ]:
# tell dlt (and Qdrant) to create a folder with our data, 
# and the name for it will be db.qdrant
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)

In [ ]:
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

# Normalized data for the following tables:
# - zoomcamp_data: 948 row(s)

In [ ]:
# embedding model: fast-bge-small-en